# Indeed Job Scraper

## Steps

1. Get all of the URL's of EACH LISTING. for each page teh MAIN function gets, get the LINKS for that page (the job postings). For example 15 pages gets a list of 150 URLS. 
- for EACH page
    - get Mosaic job card link
    - iterate through the list of job cards
    - append list of URLS by each individual URL
    - move to next page in Indeed
2. Look through the entire URL list and process them individually.
- for each URL
- extract the parts of the page for features 
3. On each individual page, 'write a row' for the data about the job as each features.

Loop through list of URLs
for each URLs, append a list by a dictionary of its contents
dump into pd df
pd.to_pickle('filepath')

In [2]:
from bs4 import BeautifulSoup
import requests
import time
import csv
from time import sleep
from random import randint
from datetime import datetime
import pandas as pd

In [3]:
def get_url(position, location,page_count):
        if page_count > 0:
            template = 'https://www.indeed.com/jobs?q={}&l={}&start={}'
            page_count *= 10
            url = template.format(position, location,page_count)
        else:
            template = 'https://www.indeed.com/jobs?q={}&l={}'
            url = template.format(position, location)
        
        return 'http://api.scraperapi.com?api_key=f39a27865b56e74c0423ab1b9ecf1f23&url=' + url

def job_listings_generator(position, location,page_count=10):
    headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47'
}
    
    urls = [] 
    i = 0
    
    # POPULATE LIST OF URLs
    
    while i < page_count:
        
        url = get_url(position, location,i)  # create the url while passing in the position and location.
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find( id = "mosaic-zone-jobcards" )
        
        if cards is None:
            print('EMPTY URL HERE! ')
            print(url)
            time.sleep(6.8)
            continue
            
        cards = cards.find_all('a')
        cards_test = [card for card in cards if 'pagead' in card['href'] or '/rc/' in card['href']]
        
        for card in cards:
            record = card['href']


            if "pagead" in record:
                urls.append("https://www.indeed.com"+record)

            if "/rc/" in record:
                urls.append("https://www.indeed.com"+record)
        
        time.sleep(0.1)
        i += 1
    
    return urls


In [4]:
def job_info(url,title,location):
    try:
        response = requests.get(url)
    except Exception as e:
        print(str(e))
        time.sleep(5.0)
        try:
            print('trying again!')
            response = requests.get(url)
        except Exception as e:
            return {}
        
    soup = BeautifulSoup(response.text, 'html.parser')
    d = dict()
    
    
    #location
    d["location"]=location
    
    #title
    d["job_title"]=title

    
    job_info = soup.find(class_="jobsearch-ViewJobLayout-jobDisplay icl-Grid-col icl-u-xs-span12 icl-u-lg-span7" )
    
#     print(job_info)
    
    
    if job_info is None:
        d["job_desc"] = ""
#         time.sleep(6.8)
        return d
        
    cards = job_info.find_all('h2')
    
    if len(cards) == 0:
        d["job_desc"] = ""
#         time.sleep(6.8)
        return d
            

    #description
    for card in cards:
        d["job_desc"]=card.find_next('div').text

    return d

In [69]:
# urls = []
# locations = ["major metros / areas"]
# titles = ["10+ job titles"]

# titles = ["data+scientist","financial+analyst", "underwriter","chemical+engineer","physician","university+recruiter"]
titles = ["chemical+engineer"]
locations = ['New+York','Los+Angeles','Chicago','Phoenix','Raleigh',"San+Francisco","Portland", "Seattle","New+York","Los+Angeles","Austin"]

urls = []
job_information = []


for title in titles:
    for location in locations:
        urls=job_listings_generator(title,location,10)
        for job in urls:
            job_information.append(job_info(job,title,location))
            
            time.sleep(0.1)
            
        print(f"finished {location} listings for {title}!")

EMPTY URL HERE! 
http://api.scraperapi.com?api_key=f39a27865b56e74c0423ab1b9ecf1f23&url=https://www.indeed.com/jobs?q=chemical+engineer&l=New+York&start=90
EMPTY URL HERE! 
http://api.scraperapi.com?api_key=f39a27865b56e74c0423ab1b9ecf1f23&url=https://www.indeed.com/jobs?q=chemical+engineer&l=New+York&start=90
finished New+York listings for chemical+engineer!


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [56]:
df = pd.DataFrame(job_information)
df.head(10)
df_fa = df[:10]
df_fa['job_desc'][0]

"Founded in 1981, IEC is a California apartment investment firm. The firm invests in value-add apartment projects in California’s best markets. IEC is currently investing a $300m discretionary fund which includes endowments, pension funds, foundations and family offices. Since inception, the firm has acquired and sold over 100 projects in its core locations.For more information visit: www.interstateequities.comDescription: Interstate Equities Corporation is seeking to expand its Analyst group. As an integral member of our team, Analysts should expect a generalist approach. The primary responsibilities include; supporting the firms’ acquisitions, dispositions, capital formation and investor relations efforts. Working closely with senior executives and other team members, Analysts are encouraged to assume increasing responsibility.Responsibilities: The primary responsibilities for the role include:· Supporting the acquisition process through detailed operational budgeting, assessing mark

In [82]:
df = pd.DataFrame(job_information)
df = df.dropna()
df.info()
df.tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134 entries, 0 to 133
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   location   134 non-null    object
 1   job_title  134 non-null    object
 2   job_desc   134 non-null    object
dtypes: object(3)
memory usage: 4.2+ KB


,location,job_title,job_desc
129,New+York,chemical+engineer,Company Description\nThe Bountiful Company is ...
130,New+York,chemical+engineer,
131,New+York,chemical+engineer,Company Overview\nNexGen Power Systems designs...
132,New+York,chemical+engineer,
133,New+York,chemical+engineer,Overview:\n\nThe process development specialis...


In [83]:
# df.to_pickle(os.getcwd()+'/webscraped_pickle')
df.to_csv('chem_scrape')

In [84]:
df6 = pd.read_csv('chem_scrape')

In [85]:
df6 = df6.dropna()
# df6=df6.drop(columns = "Unnamed: 0")

df6.head()
df6.info()
df6.to_csv('chem_scrape')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  82 non-null     int64 
 1   location    82 non-null     object
 2   job_title   82 non-null     object
 3   job_desc    82 non-null     object
dtypes: int64(1), object(3)
memory usage: 3.2+ KB


In [43]:
df6 = pd.read_csv('ds_scrape')

In [86]:
df6

,Unnamed: 0,location,job_title,job_desc
0,0,New+York,chemical+engineer,Ball Corporation manufactures aluminum packagi...
1,1,New+York,chemical+engineer,Job Description:\nKodak is Hiring!\n\nWe are l...
2,2,New+York,chemical+engineer,Requirments:· Responsible for various aspects ...
4,4,New+York,chemical+engineer,Conamix is seeking a Research Engineer to inno...
6,6,New+York,chemical+engineer,FULL-TIME OR CONTRACT POSITION\n\nDESCRIPTION:...
...,...,...,...,...
127,127,New+York,chemical+engineer,\nNew York City Residency is not required for ...
128,128,New+York,chemical+engineer,B4C Process Engineer\nWashington Mills Electro...
129,129,New+York,chemical+engineer,Company Description\nThe Bountiful Company is ...
131,131,New+York,chemical+engineer,Company Overview\nNexGen Power Systems designs...


In [50]:
df6.to_csv('ds_scrape')